In [1]:
!pip install pygame

In [1]:
import pygame
import os

pygame 2.5.2 (SDL 2.28.3, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [16]:
import pygame
import os
import json

# Pygame hasieratu
pygame.init()

# Irudiak karpeta batetik irakurri eta gordetzeko funtzioa
def load_images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            images.append((filename, pygame.image.load(os.path.join(folder_path, filename))))
    return images

# Irudien karpeta adierazi eta kargatu
image_folder = "images"
image_list = load_images(image_folder)
total_images = len(image_list)
current_image_index = 0

# Pantailaren tamaina adierazi (irudien resolution ikusi eta aldatu)
window_width = 1765
window_height = 993
screen = pygame.display.set_mode((window_width, window_height))

# Irudien tamaina eskalatzeko funtzioa
def calculate_scale_factor(image):
    image_width, image_height = image.get_size()
    return min(window_width / image_width, window_height / image_height)

# Irudiak eguneratu/aldatzeko funtzioa (EZKER/ESKUBI FLETXAK ERABILI ALDATZEKO IRUDIA)
def update_display():
    screen.fill((255, 255, 255))

    scaled_image = pygame.transform.scale(image_list[current_image_index][1],
                                          (int(image_list[current_image_index][1].get_width() * calculate_scale_factor(image_list[current_image_index][1])),
                                           int(image_list[current_image_index][1].get_height() * calculate_scale_factor(image_list[current_image_index][1]))))
    screen.blit(scaled_image, (0, 0))
    pygame.display.update()

# botoien koordenatuak gordeko diren JSON fitxategia sortu
json_filename = "button_coordinates.json"
button_coordinates_data = {}

top_left_coordinates = None

font = pygame.font.Font(None, 36)
instruction_text = font.render("Change gamescreens using LEFT/RIGHT ARROWS. Firstly, click the TOP-LEFT CORNER of a button, then click the BOTTOM-RIGHT CORNER of it.", True, (255, 0, 0))

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if top_left_coordinates is None:
                # botoiaren goi ezker puntua gorde 
                x, y = event.pos
                top_left_coordinates = (x, y)
            else:
                # botoiaren behe eskuin puntua gorde
                x, y = event.pos
                bottom_right_coordinates = (x, y)

                # gorde botoiaren koordenatuak
                if image_list[current_image_index][0] not in button_coordinates_data:
                    button_coordinates_data[image_list[current_image_index][0]] = []

                button_coordinates_data[image_list[current_image_index][0]].append({
                    "top_left": {"x": top_left_coordinates[0], "y": top_left_coordinates[1]},
                    "bottom_right": {"x": bottom_right_coordinates[0], "y": bottom_right_coordinates[1]}
                })

                # reset egin botoi berri bat gorde ahal izateko
                top_left_coordinates = None

        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:  # Aurreko irudia
                current_image_index = (current_image_index - 1) % total_images
                update_display()
            elif event.key == pygame.K_RIGHT:  # Hurrengo irudia
                current_image_index = (current_image_index + 1) % total_images
                update_display()

    # Eguneratu irudia
    update_display()
    #Instrukzioak pantailan agertzeko
    screen.blit(instruction_text, (20, 20))
    pygame.display.flip()
    
# Gorde botoien koordenatuak json batean
with open(json_filename, 'w') as json_file:
    json.dump(button_coordinates_data, json_file, indent=2)

pygame.quit()
